# Process Environment

## How the main function is called when a program is executed?

When a C program is executed by the kernel, —a special start-up routine is called before the main function is called. 
The executable program file specifies this routine as the starting address for the program; this is set up by the link editor when it is invoked by the C compiler. This start-up routine takes values from the kernel—the command-line arguments and the environment

## Process Termination

Normal Termination
1. Return from main
2. Calling exit
3. Calling _exit or _Exit
4. Return of the last thread from its start routine (Section 11.5)
5. Calling pthread_exit (Section 11.5) from the last thread

Abnormal Termination
1. Calling abort (Section 10.17)
2. Receipt of a signal (Section 10.2)
3. Response of the last thread to a cancellation request


### Exit functions
``` c

#include <stdlib.h>
void exit(int status);
void _Exit(int status);
#include <unistd.h>
void _exit(int status);

```

In [3]:
! echo $? # print exit status of last executed program

0


### atexit Function

With ISO C, a process can register at least 32 functions that are automatically called by
exit. These are called exit handlers and are registered by calling the atexit function.

```c
#include <stdlib.h>
int atexit(void (*func)(void));
```

This declaration says that we pass the address of a function as the argument to atexit.
When this function is called, it is not passed any arguments and is not expected to
return a value. The exit function calls these functions in reverse order of their
registration. Each function is called as many times as it was registered.

<img src="images/exit.png"  width="900" height="400">

## Command line arguments

## Environment list

Each program is also passed an environment list. Like the argument list, the
environment list is an array of character pointers, with each pointer containing the
address of a null-terminated C string. The address of the array of pointers is contained
in the global variable environ:

```c
extern char **environ;
```

Access to specific environment variables is normally through the getenv
and putenv functions,instead of through the environ
variable. But to go through the entire environment, the environ pointer must be used.

## Memory Layout of a C Program

<img src="images/mm.png"  width="600" height="400">

- **Text segment**, consisting of the machine instructions that the CPU executes.
  Usually, the text segment is sharable so that only a single copy needs to be in
  memory for frequently executed programs, such as text editors, the C compiler, 
  the shells, and so on. Also, the text segment is often read-only, to prevent a
  program from accidentally modifying its instructions.
  
- **Initialized data segment**, usually called simply the data segment, containing
  variables that are specifically initialized in the program.
  
- **Uninitialized data segment**, often called the ‘‘bss’’ segment, named after an
  ancient assembler operator that stood for ‘‘block started by symbol.’’ Data in
  this segment is initialized by the kernel to arithmetic 0 or null pointers before the
  program starts executing.
  
- **Stack**, where automatic variables are stored, along with information that is saved
  each time a function is called. Each time a function is called, the address of
  where to return to and certain information about the caller’s environment, such
  as some of the machine registers, are saved on the stack. The newly called
  function then allocates room on the stack for its automatic and temporary
  variables. This is how recursive functions in C can work. Each time a recursive
  function calls itself, a new stack frame is used, so one set of variables doesn’t
  interfere with the variables from another instance of the function.
  
- **Heap**, where dynamic memory allocation usually takes place. Historically, the
  heap has been located between the uninitialized data and the stack.

### The size(1) command reports the sizes (in bytes) of the text, data, and bss segments.

In [6]:
! size /home/chandan/CHANDAN/Unix/apue.3e-master/intro/hello

   text	   data	    bss	    dec	    hex	filename
   1341	    548	      4	   1893	    765	/home/chandan/CHANDAN/Unix/apue.3e-master/intro/hello


## Shared Libraries

Shared libraries remove the common library routines
from the executable file, instead maintaining a single copy of the library routine
somewhere in memory that all processes reference. This reduces the size of each
executable file but may add some runtime overhead, either when the program is first
executed or the first time each shared library function is called. Another advantage of
shared libraries is that library functions can be replaced with new versions without
having to relink edit every program that uses the library (assuming that the number and
type of arguments haven’t changed).

gcc -static hello1.c #prevent gcc from using shared libraries

gcc hello1.c        #gcc defaults to use shared libraries

## Memory Allocation

ISO C specifies three functions for memory allocation:
1. malloc, which allocates a specified number of bytes of memory. The initial
   value of the memory is indeterminate.
2. calloc, which allocates space for a specified number of objects of a specified
   size. The space is initialized to all 0 bits.
3. realloc, which increases or decreases the size of a previously allocated area.
   When the size increases, it may involve moving the previously allocated area
   somewhere else, to provide the additional room at the end. Also, when the size
   increases, the initial value of the space between the old contents and the end of
   the new area is indeterminate.
   
```c
#include <stdlib.h>
void *malloc(size_t size);
void *calloc(size_t nobj, size_t size);
void *realloc(void *ptr, size_t newsize);
//All three return: non-null pointer if OK, NULL on error
void free(void *ptr);
```

>The allocation routines are usually implemented with the sbrk(2) system call. This
system call expands (or contracts) the heap of the process. (Refer to Figure 7.6.) A
sample implementation of malloc and free is given in Section 8.7 of Kernighan and
Ritchie [1988].

### Memory leak
Other possible errors that can be fatal are freeing a block that was already freed and
calling free with a pointer that was not obtained from one of the three alloc
functions. If a process calls malloc but forgets to call free, its memory usage will
continually increase; this is called leakage. If we do not call free to return unused
space, the size of a process’s address space will slowly increase until no free space is left.
During this time, performance can degrade from excess paging overhead.

## Environment variables

```c
#include <stdlib.h>
int putenv(char *str);
//Returns: 0 if OK, nonzero on error
int setenv(const char *name, const char *value, int rewrite);
int unsetenv(const char *name);
//Both return: 0 if OK, −1 on error
```

## setjmp and longjmp functions

In C, we can’t goto a label that’s in another function. Instead, we must use the setjmp
and longjmp functions to perform this type of branching.

## Limitation of resource

Every process has a set of resource limits, some of which can be queried and changed by
the getrlimit and setrlimit functions.

```c
#include <sys/resource.h>
int getrlimit(int resource, struct rlimit *rlptr);
int setrlimit(int resource, const struct rlimit *rlptr);
//Both return: 0 if OK, −1 on error
```
Each call to these two functions specifies a single resource and a pointer to the
following structure:
```c
struct rlimit {
rlim_t rlim_cur;
rlim_t rlim_max;
};
```

Three rules govern the changing of the resource limits.
1. A process can change its soft limit to a value less than or equal to its hard limit.
2. A process can lower its hard limit to a value greater than or equal to its soft
   limit. This lowering of the hard limit is irreversible for normal users.
3. Only a superuser process can raise a hard limit.

>The resource limits affect the calling process and are inherited by any of its children.
This means that the setting of resource limits needs to be built into the shells to affect all
our future processes. Indeed, the Bourne shell, the GNU Bourne-again shell, and the
Korn shell have the built-in ulimit command, and the C shell has the built-in limit
command. (The umask and chdir functions also have to be handled as shell built-ins.)
